In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls

Patient 2 [label YES, NO, MISSING] - GPT-Labeled-LLM_Note (Bolanle).csv
Patient 2 [label YES, NO, MISSING] - GPT-Labled-Transcript (Waris).csv
Patient Labelling - GPT-Labled-Transcript.csv
Patient Labelling - Manual-Labled-Transcript.csv
Patient2_Scoring.ipynb
Scoring Presence Accuracy.ipynb


In [3]:
gpt_transcript = pd.read_csv("Patient Labelling - GPT-Labled-Transcript.csv")
manual_transcript = pd.read_csv("Patient Labelling - Manual-Labled-Transcript.csv")

In [4]:
# get rid of whitespace around text

gpt_transcript.columns = gpt_transcript.columns.str.strip()
gpt_transcript = gpt_transcript.applymap(lambda x: x.strip() if isinstance(x, str) else x)

manual_transcript.columns = manual_transcript.columns.str.strip()
manual_transcript = manual_transcript.applymap(lambda x: x.strip() if isinstance(x, str) else x)


/var/folders/c5/k58v_7fn31j_jp5cz2ggmbs00000gn/T/ipykernel_68899/1985135469.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gpt_transcript = gpt_transcript.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/c5/k58v_7fn31j_jp5cz2ggmbs00000gn/T/ipykernel_68899/1985135469.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  manual_transcript = manual_transcript.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [5]:
gpt_transcript.head()

,Timestamp,Script,Placenta Previa,Placenta Accreta Spectrum or concern,Multiple Gestation,Polyhydramnios,Large for Gestational Age,Fibroids,Preeclampsia,HELLP,Anticoagulation use,Baseline Laboratory Values - Hemoglobin/Hematocrit,Baseline Laboratory Values - Platelet Count,"Baseline Laboratory Values - Coagulopathy (PTT, PT/INR)"
0,[00:00.000 --> 00:02.000,Hello.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[00:02.000 --> 00:04.000,"Hi, good morning.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[00:04.000 --> 00:07.000,This is Dr. Smith from the Bergenman Women's H...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[00:07.000 --> 00:08.000,Dr. Canis-Physion team.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[00:08.000 --> 00:11.000,I'm looking for Ms. Mary Blank.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
manual_transcript.head()

,Timestamp,Script,Placenta Previa,Placenta Accreta Spectrum or concern,Multiple Gestation,Polyhydramnios,Large for Gestational Age,Fibroids,Preeclampsia,HELLP,Anticoagulation use,Baseline Laboratory Values - Hemoglobin/Hematocrit,Baseline Laboratory Values - Platelet Count,"Baseline Laboratory Values - Coagulopathy (PTT, PT/INR)"
0,[00:00.000 --> 00:02.000,Hello.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[00:02.000 --> 00:04.000,"Hi, good morning.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[00:04.000 --> 00:07.000,This is Dr. Smith from the Bergenman Women's H...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[00:07.000 --> 00:08.000,Dr. Canis-Physion team.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[00:08.000 --> 00:11.000,I'm looking for Ms. Mary Blank.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
(set(gpt_transcript.columns) - set(["Timestamp", "Script"])) == (set(manual_transcript.columns) - set(["Timestamp", "Script"]))

True

In [15]:
assert manual_transcript.shape == gpt_transcript.shape

In [26]:
import math
correct_count = 0
total_count = 0

wrong_labels = []
total_labels = []

for (index_g, row_g), (index_m, row_m) in zip(gpt_transcript.iterrows(), manual_transcript.iterrows()):
    for feature_j in set(gpt_transcript.columns) - set(["Timestamp", "Script"]):
        if (pd.isna(row_g[feature_j]) and pd.isna(row_m[feature_j])) or row_g[feature_j] == row_m[feature_j]:
            correct_count += 1
        else:
            wrong_labels.append(feature_j)
            print(row_g[feature_j], row_m[feature_j], feature_j, index_g)
            
        total_count += 1
        total_labels.append(feature_j)


print(correct_count/total_count, correct_count, total_count)

print(len(set(wrong_labels)), len(set(total_labels)))

nan Negative Large for Gestational Age 51
nan Negative Large for Gestational Age 52
0.9995224450811844 4186 4188
1 12
